In [9]:
import os

In [10]:
import sys

In [11]:
import random

In [12]:
import nltk

In [13]:
from nltk.corpus import stopwords

In [14]:
from nltk import FreqDist

In [15]:
from nltk import word_tokenize

In [16]:
!pip install langdetect

In [17]:
import langdetect

In [18]:
from langdetect import detect

In [19]:
import re

In [20]:
from nltk.metrics import precision, recall, f_measure

In [21]:
from nltk.classify import NaiveBayesClassifier

In [22]:
from nltk.classify.util import accuracy

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
from sklearn.model_selection import cross_val_score

In [38]:
def document_features(document, word_features, tfidf_vectorizer):
    doc_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in doc_words)
    return features
    #tfidf score calculation
    tfidf_scores = tfidf_vectorizer.transform([' '.join(document)])
    for feature, score in zip(word_features, tfidf_scores.data):
        features[feature] = score
    return features
    #negation representation
    for x in range(len(document)):
        if x > 0 and document[x-1].lower() in ['not', 'no', 'never', 'neither', 'nobody', 'none', 'nor', 'nothing', 'nowhere']:
            features['negation_{}'.format(document[x].lower())] = True
    return features
    #POS tags
    pos_tags = nltk.pos_tag(document)
    for tag in pos_tags:
        features['pos_tag_{}'.format(tag[1])] = True
    return features

In [39]:
#finding the top 20 used words
def get_top_words(emaildocs, num_words=20):
    all_words = FreqDist(word for tokens, _ in emaildocs for word in tokens)
    top_words = [word for word, _ in all_words.most_common(num_words)]
    return top_words

In [40]:
#making sure tokens are English
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [41]:
#filtering tokens that aren't real words
def filter_gibberish(tokens, min_word_length=3):
    return [word for word in tokens if len(word) >= min_word_length and is_english(word)]

In [42]:
#split tokens that seem to be combined words
def split_combined_words(tokens):
    return [subtoken.lower() for token in tokens for subtoken in re.findall(r'\b\w+\b', token)]

In [46]:
# function to read spam and ham files, train and test a classifier 
def processspamham(dirPath,limitStr):
  # convert the limit argument from a string to an int
  limit = int(limitStr)
  
  # start lists for spam and ham email texts
  hamtexts = []
  spamtexts = []
  os.chdir(dirPath)
  # process all files in directory that end in .txt up to the limit
  #    assuming that the emails are sufficiently randomized
  for file in os.listdir("./spam"):
    if (file.endswith(".txt")) and (len(spamtexts) < limit):
      # open file for reading and read entire file into a string
      f = open("./spam/"+file, 'r', encoding="latin-1")
      spamtexts.append (f.read())
      f.close()
  for file in os.listdir("./ham"):
    if (file.endswith(".txt")) and (len(hamtexts) < limit):
      # open file for reading and read entire file into a string
      f = open("./ham/"+file, 'r', encoding="latin-1")
      hamtexts.append (f.read())
      f.close()
  
  # print number emails read
  print ("Number of spam files:",len(spamtexts))
  print ("Number of ham files:",len(hamtexts))
  print
  
  # create list of mixed spam and ham email documents as (list of words, label)
  emaildocs = []
  # add all the spam
  for spam in spamtexts:
    tokens = nltk.word_tokenize(spam)
    emaildocs.append((tokens, 'spam'))
  # add all the regular emails
  for ham in hamtexts:
    tokens = nltk.word_tokenize(ham)
    emaildocs.append((tokens, 'ham'))
  
  # randomize the list
  random.shuffle(emaildocs)



  #filtering out gibberish and combined words
  for spam in spamtexts:
        if is_english(spam):  # Filter out non-English text
            tokens = nltk.word_tokenize(spam)
            #filter out potential gibberish and combined words
            filtered_tokens = split_combined_words(filter_gibberish(tokens))
            emaildocs.append((filtered_tokens, 'spam'))
    
  for ham in hamtexts:
        if is_english(ham):  # Filter out non-English text
            tokens = nltk.word_tokenize(ham)
            # Filter out short words and non-English words (potential gibberish)
            filtered_tokens = split_combined_words(filter_gibberish(tokens))
            emaildocs.append((filtered_tokens, 'ham'))

    # possibly filter tokens
  stop_words = set(stopwords.words('english'))
  emaildocs = [([word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words], label) for tokens, label in emaildocs]
  
  # continue as usual to get all words and create word features
  all_words = FreqDist(word for tokens, _ in emaildocs for word in tokens)

  #find the top 20 most used words
  top_words = [word for word, _ in all_words.most_common(20)]

  # feature sets from a feature definition function
  word_features = list(all_words.keys())[:3000]
  all_texts = [' '.join(tokens) for tokens, _ in emaildocs]
  tfidf_vectorizer = TfidfVectorizer(stop_words='english')
  tfidf_vectorizer.fit(all_texts)
  featuresets = [(document_features(tokens, word_features, tfidf_vectorizer), label) for tokens, label in emaildocs]

  #tfidf vectorizer
 
  emaildocs_tfidf = [([word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words], label) for tokens, label in emaildocs]
  all_words_tfidf = FreqDist(word for tokens, _ in emaildocs_tfidf for word in tokens)
  top_words_tfidf = [word for word, _ in all_words_tfidf.most_common(20)]
  featuresets_tfidf = [(document_features(tokens, top_words_tfidf, tfidf_vectorizer), label) for tokens, label in emaildocs_tfidf]
  
    # train classifier and show performance in cross-validation
  num_folds = 5
  subset_size = int(len(featuresets) / num_folds)
  precision_scores = []
  recall_scores = []
  f_measure_scores = []

  for i in range(num_folds):
        testing_set = featuresets[i * subset_size:][:subset_size]
        training_set = featuresets[:i * subset_size] + featuresets[(i + 1) * subset_size:]

        classifier = NaiveBayesClassifier.train(training_set)

        # Evaluate the classifier on the testing set
        reference_set = set(label for _, label in testing_set)
        test_set = set(classifier.classify(features) for features, _ in testing_set)

        precision_scores.append(precision(reference_set, test_set))
        recall_scores.append(recall(reference_set, test_set))
        f_measure_scores.append(f_measure(reference_set, test_set))

  #Negation representation
        for examples, label in testing_set[:4]:
            print("Negation Representation for '{}': ".format(label))
            for feature, value in examples.items():
                if feature.startswith('negation_') and value:
                    print(feature)
        #POS tags
        for examples, label in testing_set[:4]:
            print("POS Tag Features for '{}': ".format(label))
            for feature, value in examples.items():
                if feature.startswith('pos_tag_') and value:
                    print(feature)
    
  #calculate the average scores
  avg_precision = sum(precision_scores) / num_folds
  avg_recall = sum(recall_scores) / num_folds
  avg_f_measure = sum(f_measure_scores) / num_folds

  #prints the average scores
  print('Avg Precision = ', avg_precision)
  print('Avg Recall = ', avg_recall)
  print('Avg F-Measure = ', avg_f_measure)
  

  #prints the top 20 words
  print('Top Words: ')
  for word in top_words:
      print(f" - {word}")
  # print a few token lists
  for email in emaildocs[:4]:
    print (email)
    print

In [47]:
processspamham(r'C:\Users\antho\OneDrive\Documents\Syracuse\664 - Natural Language Processing\Homework\Final Project\EmailSpamCorpora\corpus', '10')

Number of spam files: 10
Number of ham files: 10


NameError: name 'SklearnNaiveBayesClassifier' is not defined